In [ ]:
import xarray
import numpy as np

global mask_value 
mask_value = -1


In [ ]:
def check_I4_I5_pair(I4_filename):
    return

def read_I4_I5_data(I4_filename,I5_filename):
    '''
    notice that we want to mask bad quality data out
    '''

    return


In [ ]:
'''
we will perform a 4-step standard fire algorithm
based on one of my old paper: https://www.sciencedirect.com/science/article/pii/S0034425717302870
1. day/time confirming 
2. cloud 
3. candidate mask: spectral filter & spatial filter
4. contextual analysis
'''

def create_daynight_mask(I4_arr,I5_arr):
    return

def create_cloud_mask(I4_arr,I5_arr,daynight_mask):
    return

def create_fire_candidate_mask(I4_arr,I5_arr,cloud_mask):
    return

def contextural_analysis():
    return